In [1]:
from config import CURRENT_PERIOD
from sp_soccer_lib import create_team_df_dict, championship_teams, no_draw_frequencies
from sp_soccer_lib.handout_helpers import style, get_country_header, make_link
from sp_soccer_lib.championships import team_stats, load_country
import math

In [2]:
def simulate_team(
    team: str,
    country_team_df,
    period: str,
    bet_progr: list,
    threshold: int,
    bet_span: int
):
    team_matches_df = country_team_df[team]
    team_matches_df = team_matches_df[team_matches_df['period'] == period]
    # print(team_matches_df.head())

    team_bet = 0
    team_wins = 0

    next_bet = 0
    for _, match in team_matches_df.iterrows():
        # print(row)
        # print('--------------------------------')
        odds = 3 if math.isnan(match["B365D"]) else match["B365D"]

        # print(type(odds))
        # print(f'Match is {match["HomeTeam"]}-{match["AwayTeam"]}. Draw gives {odds}. Result for {team} is {match["result"]}')
        if next_bet:
            team_bet = team_bet + bet_progr[next_bet-1]
            # print(f'!!! This is bet No {next_bet}. Betting {bet_progr[next_bet-1]} Eur.  !!!')
        else:
            # print('!!! No bet this time !!!')
            pass
        if match['FTR'] == 'D':
            if next_bet:
                # print(f'Colourful Ballons!!! We retrieve {bet_progr[next_bet - 1] * odds}')
                team_wins = team_wins + bet_progr[next_bet - 1] * odds
            next_bet = 0
        else:
            if match['count_no_draw'] >= threshold and match['count_no_draw'] < threshold + bet_span:
                next_bet = match['count_no_draw'] - threshold + 1
            else:
                next_bet = 0

                # print('!!!!!BET!!!!!!')
        # if row['count_no_draw'] == threshold:
        #     next_bet = 1

    # print(f'{team} bet for {period}: {team_bet}')
    # print(f'{team} wins for {period}: {team_wins}')
    return team_bet, team_wins


In [8]:
country = 'greece'
# period = '2122'
# threshold = 6
# bet_span = 10
# bet_progr = [2,4,8, 16, 32, 64, 128, 256, 512, 1024]
bet_progr = [2,4,6, 9, 13, 20, 30, 45, 68, 103]
# bet_progr = [2,4,8, 16, 32, 64, 128, 256, 512, 1024]
# bet_progr = [2,4,8, 16, 32, 64, 128, 256, 512, 1024]

df = load_country(country)
# print(df.head(2))
team_df = create_team_df_dict(df)
# print(team_df)
stats_df = team_stats(team_df)
# print(stats_df.head(2))
total_bet = 0
total_wins = 0
team_result = []
period_result = []
for country in ['greece', 'italy']:
    for threshold in [3, 4, 5, 6]:
        for bet_span in [3, 4, 5, 6, 7, 8, 9, 10]:
            for period in ['2021', '2122']:
                for _, team_row in stats_df.iterrows():
                    print(team_row.name)
                    team = team_row.name
                    team_bet, team_wins = simulate_team(team, team_df, period, bet_progr, threshold, bet_span)
                    total_bet = total_bet + team_bet
                    total_wins = total_wins + team_wins
                    team_result.append({'country': country, 'period': period, 'threshold': threshold, 'span': bet_span, 'team': team, 'bet': team_bet, 'wins': team_wins})

                print(f'Total bet: {total_bet}')
                print(f'Total wins: {total_wins}')
                period_result.append({'country': country, 'period': period, 'threshold': threshold, 'span': bet_span, 'total_bet': total_bet, 'total_wins': total_wins})
                total_bet = 0
                total_wins = 0
print(team_result)
print(period_result)

2022-09-19 14:32:10.537 | INFO     | sp_soccer_lib:period_stats:96 - Made Aris 2122 Adjustment


Panathinaikos
PAOK
Aris
AEK
Atromitos
Olympiakos
Volos NFC
Panetolikos
Asteras Tripolis
OFI Crete
Giannina
Lamia
Levadeiakos
Ionikos
Apollon
Apollon Apollon
Kerkyra
Larisa
Panionios
Platanias
Xanthi
Total bet: 400
Total wins: 332.80000000000007
Panathinaikos
PAOK
Aris
AEK
Atromitos
Olympiakos
Volos NFC
Panetolikos
Asteras Tripolis
OFI Crete
Giannina
Lamia
Levadeiakos
Ionikos
Apollon
Apollon Apollon
Kerkyra
Larisa
Panionios
Platanias
Xanthi
Total bet: 420
Total wins: 353.79999999999995
Panathinaikos
PAOK
Aris
AEK
Atromitos
Olympiakos
Volos NFC
Panetolikos
Asteras Tripolis
OFI Crete
Giannina
Lamia
Levadeiakos
Ionikos
Apollon
Apollon Apollon
Kerkyra
Larisa
Panionios
Platanias
Xanthi
Total bet: 580
Total wins: 608.1999999999999
Panathinaikos
PAOK
Aris
AEK
Atromitos
Olympiakos
Volos NFC
Panetolikos
Asteras Tripolis
OFI Crete
Giannina
Lamia
Levadeiakos
Ionikos
Apollon
Apollon Apollon
Kerkyra
Larisa
Panionios
Platanias
Xanthi
Total bet: 600
Total wins: 482.95000000000005
Panathinaikos
PAOK
Ar

In [9]:
import pandas as pd
pd.DataFrame.from_dict(team_result).to_excel('team.xlsx')
pd.DataFrame.from_dict(period_result).to_excel('period.xlsx')